In [1]:
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import time
import math
from collections import OrderedDict
import random
from torchsummary import summary
from torchvision import transforms
from torch.utils.data import Dataset
import sys
import torch
from tqdm import trange
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mne
from sklearn.preprocessing import StandardScaler
import logging
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence
import statistics
import torch.optim.lr_scheduler as lr_scheduler
from scipy.special import softmax
from sklearn.model_selection import StratifiedKFold
from scipy.signal import welch
from scipy.stats import entropy

c:\Users\admin\anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
logging.getLogger('mne').setLevel(logging.WARNING)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import import_ipynb
#from Model import net
from Dataloader2 import EEGDataset

importing Jupyter notebook from Dataloader2.ipynb


In [3]:
MNE_Data = EEGDataset(root_dir=r"C:\Users\admin\Desktop\TEST")
labels = MNE_Data.labels
#MNE_Data = EEGDataset(root_dir=r"D:\TEST MNE")
test_path = r"C:\Users\admin\Desktop\TEST\amirifateme.fif"
test_class = 0

In [4]:
patient = MNE_Data.__getitem__(0)[0]
eeg_data = patient[120]
patient = np.random.randn(19400, 19, 3000)

In [5]:
sf = 300  # Hz, replace with your actual sampling frequency

# Function to calculate spectral entropy
def spectral_entropy(signal, sf, method='welch', normalize=True):
    # Compute the Power Spectral Density (PSD) using Welch's method
    freqs, psd = welch(signal, sf)
    
    # Normalize the PSD to get a probability distribution, if required
    if normalize:
        psd_norm = psd / np.sum(psd)
        return entropy(psd_norm)
    else:
        return entropy(psd)

features = []
for window in patient:
    # Initialize an array to store the spectral entr
    # opy for each channel
    spectral_entropy_features = np.zeros((window.shape[0],))

    # Calculate spectral entropy for each channel
    for i in range(window.shape[0]):
        spectral_entropy_features[i] = spectral_entropy(window[i, :], sf)

    # Output the spectral entropy features
    
    features.append(spectral_entropy_features)

In [6]:
np.shape(features)

(19400, 19)

In [7]:
y = np.random.randint(0, 4, size=(19400))
y

array([3, 2, 2, ..., 1, 1, 1])

In [9]:
from sklearn.tree import DecisionTreeClassifier

# Assuming 'X' is your feature matrix with shape (194, n_features)
# and 'y' is your label vector with the 4 classes encoded as 0, 1, 2, 3

# Initialize the list of decision trees
trees = []

# Create and train a decision tree for each class
for class_label in range(4):
    # Create a binary label vector where the current class is 1 and all others are 0
    y_binary = (y == class_label).astype(int)
    
    # Initialize the decision tree classifier
    tree = DecisionTreeClassifier(random_state=42)
    
    # Train the classifier
    tree.fit(features, y_binary)
    
    # Store the trained tree
    trees.append(tree)

# Function to predict using the sequence of decision trees
def predict(features, trees):
    predictions = []
    
    for sample in features:
        for i, tree in enumerate(trees):
            # Predict with the current tree
            prediction = tree.predict([sample])
            
            # If the tree classifies the sample as the current class, break and assign that class
            if prediction == 1:
                predictions.append(i)
                break
        else:
            # If none of the trees classify the sample as their class, assign a default class
            predictions.append(0)  # You can choose how to handle this case
    
    return predictions

# Example usage
# X_test is your test feature matrix
#predictions = predict(features, trees)


In [15]:
for i in range(40):
    tree = DecisionTreeClassifier(max_depth=i+1)
    tree.fit(features, y)
    predictions = tree.predict(features)
    print(f"tree depth: {i+1}")
    print(np.mean(predictions == y))

tree depth: 1
0.25257731958762886
tree depth: 2
0.25304123711340204
tree depth: 3
0.25876288659793817
tree depth: 4
0.26123711340206185
tree depth: 5
0.26798969072164947
tree depth: 6
0.2754639175257732
tree depth: 7
0.28304123711340207
tree depth: 8
0.2910824742268041
tree depth: 9
0.30355670103092786
tree depth: 10
0.3154639175257732
tree depth: 11
0.3306185567010309
tree depth: 12
0.3504639175257732
tree depth: 13
0.3789175257731959
tree depth: 14
0.4060824742268041
tree depth: 15
0.4367525773195876
tree depth: 16
0.4692783505154639
tree depth: 17
0.5017525773195877
tree depth: 18
0.5364432989690722
tree depth: 19
0.5726288659793815
tree depth: 20
0.6094845360824742
tree depth: 21
0.6427835051546392
tree depth: 22
0.6752577319587629
tree depth: 23
0.7079896907216495
tree depth: 24
0.7405670103092783
tree depth: 25
0.7751546391752577
tree depth: 26
0.8060309278350516
tree depth: 27
0.8347422680412371
tree depth: 28
0.8618556701030928
tree depth: 29
0.8858762886597938
tree depth: 30
0

0.9695876288659794